# OMERO Micro-SAM Workflow

In [1]:
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline,
)

from omero_annotate_ai.core.config import load_config_from_yaml
import os
from pathlib import Path

print(f"omero-annotate-ai version: {omero_annotate_ai.__version__}")

try:
    import ezomero
    print("OMERO functionality: Available")
except ImportError:
    print("OMERO functionality: Install with: pip install -e .[omero]")

try:
    import keyring
    print("Keyring support: Available")
except ImportError:
    print("Keyring support: Not available")

omero-annotate-ai version: 0.1.0
OMERO functionality: Available
Keyring support: Available


## OMERO Connection

In [ ]:
conn_widget = create_omero_connection_widget()
conn_widget.display()

📄 Loaded configuration from connection history: root@localhost
🔐 Password loaded from keychain (no expiration)
🔐 Password loaded from keychain (no expiration)


In [3]:
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established")

print(f"Connected as: {conn.getUser().getName()}")
print(f"Group: {conn.getGroupFromContext().getName()}")

Connected as: root
Group: system


In [4]:
# Create the enhanced configuration widget with table management
workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

## Pipeline Execution

In [5]:
config = load_config_from_yaml(
    "C:\\Users\\Maarten\\AppData\Local\Temp\\omero_annotations_m_14q2cq\\annotation_config.yaml"
)

config.validate()

print(f"Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"Training Set: {config.training.trainingset_name}")
print(f"Model: {config.micro_sam.model_type}")
print(f"Output: {config.batch_processing.output_folder}")

if config.patches.use_patches:
    print(f"Patches: {config.patches.patches_per_image} per image ({config.patches.patch_size[0]}×{config.patches.patch_size[1]})")

print(f"Scope: {'All images' if config.training.segment_all else f'{config.training.train_n} training + {config.training.validate_n} validation'}")

Container: project (ID: 201)
Training Set: micro_sam_foci_test_20250815_090353
Model: vit_b_lm
Output: C:\Users\Maarten\AppData\Local\Temp\omero_annotations_m_14q2cq
Patches: 1 per image (256×256)
Scope: 1 training + 1 validation


In [9]:
pipeline = create_pipeline(config, conn)

container = conn.getObject(config.omero.container_type.capitalize(), config.omero.container_id)
if container is None:
    raise ValueError(f"{config.omero.container_type} with ID {config.omero.container_id} not found")

print(f"Container: {container.getName()}")

images_list = pipeline.get_images_from_container()
print(f"Images to process: {len(images_list)}")

for i, img in enumerate(images_list[:3]):
    print(f"  {i+1}. {img.getName()} (ID: {img.getId()})")
if len(images_list) > 3:
    print(f"  ... and {len(images_list) - 3} more")

Container: foci_test
📁 Loading images from project 201
📊 Found 10 images
Images to process: 10
  1. 001_MAX_ExpMP2410_005_IB10#1_2h, Position001.tif (ID: 451)
  2. 002_MAX_ExpMP2410_005_IB10#1_2h, Position002.tif (ID: 452)
  3. 009_MAX_ExpMP2410_005_IB10#1_noIR, Position001.tif (ID: 453)
  ... and 7 more


In [10]:
# Step 1: Create annotation table
table_id, images_list = pipeline.create_annotation_table(images_list)
print(f"Table created with ID: {table_id}")

# Step 2: Run annotation processing
table_id, processed_images = pipeline.run_annotation(table_id, images_list)

print(f"Completed. Processed {len(processed_images)} images")
print(f"Table ID: {table_id}")

🚀 Creating annotation table
📊 Creating table for 10 images with model: vit_b_lm
📋 Creating new tracking table: micro_sam_training_micro_sam_foci_test_20250815_090353
📋 ROI namespace: omero_annotate_ai.table.micro_sam_training_micro_sam_foci_test_20250815_090353


C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\omero\omero_functions.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).infer_objects(copy=False).astype(bool)
C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\omero\omero_functions.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).infer_objects(copy=False).astype(bool)
C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\omero\omero_functions.py:135: FutureWarning:

📋 Created tracking table 'micro_sam_training_micro_sam_foci_test_20250815_090353' with 2 units
   Container: project 201
   Table ID: 1232
object group 0
Stored configuration as annotation ID: 1233
object group 0
📊 Workflow status updated: 0/2 (0.0%) - pending
📋 Created annotation table with ID: 1232
Table created with ID: 1232
🚀 Starting annotation processing from table ID: 1232
[TABLE] Getting unprocessed units from table 1232
object group 0
📊 Workflow status updated: 0/2 (0.0%) - pending
📋 Created annotation table with ID: 1232
Table created with ID: 1232
🚀 Starting annotation processing from table ID: 1232
[TABLE] Getting unprocessed units from table 1232
📋 Found 2 unprocessed units
📋 Found 2 processing units
🔄 Processing batch 1/1
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1
📋 Found 2 unprocessed units
📋 Found 2 processing units
🔄 Processing batch 1/1
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Pro

Precompute state for files: 100%|██████████| 2/2 [00:22<00:00, 11.49s/it]
C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\core\pipeline.py:377: UserWarning: Refusing to run a QApplication with no topLevelWidgets. To run the app anyway, use `run(force=True)`
  napari.run()

C:\Users\Maarten\Documents\Github\omero_annotate_ai\src\omero_annotate_ai\core\pipeline.py:377: UserWarning: Refusing to run a QApplication with no topLevelWidgets. To run the app anyway, use `run(force=True)`
  napari.run()


Precomputation took 24.861404180526733 seconds (= 00:25 minutes)
All images have already been annotated and you have set 'skip_segmented=True'. Nothing to do.
📁 Found 2 annotation files in C:\Users\Maarten\AppData\Local\Temp\omero_annotations_m_14q2cq\annotations
🔍 Step 1: Loading label image from C:\Users\Maarten\AppData\Local\Temp\omero_annotations_m_14q2cq\annotations\seg_00000.tif
📋 Label image loaded: (256, 256), dtype: uint32
🏷️ Found 1 unique labels: [0]...
🔍 Step 2: Converting labels to ROI shapes...
✅ Created 0 ROI shapes from labels
🔍 Step 3: Uploading label file as attachment
📁 Found 2 annotation files in C:\Users\Maarten\AppData\Local\Temp\omero_annotations_m_14q2cq\annotations
🔍 Step 1: Loading label image from C:\Users\Maarten\AppData\Local\Temp\omero_annotations_m_14q2cq\annotations\seg_00000.tif
📋 Label image loaded: (256, 256), dtype: uint32
🏷️ Found 1 unique labels: [0]...
🔍 Step 2: Converting labels to ROI shapes...
✅ Created 0 ROI shapes from labels
🔍 Step 3: Upload

## Results

In [ ]:
print(f"Training Set: {config.training.trainingset_name}")
print(f"Table ID: {table_id}")
print(f"Images Processed: {len(processed_images)}")
print(f"Output: {config.batch_processing.output_folder}")

if processed_images:
    print(f"Processed Images:")
    for i, img in enumerate(processed_images[:5]):
        if hasattr(img, 'getName'):
            print(f"  {i+1}. {img.getName()} (ID: {img.getId()})")
        else:
            img_obj = conn.getObject("Image", img)
            if img_obj:
                print(f"  {i+1}. {img_obj.getName()} (ID: {img})")
    
    if len(processed_images) > 5:
        print(f"  ... and {len(processed_images) - 5} more")

In [ ]:
config_filename = f"annotation_config_{config.training.trainingset_name}.yaml"
config_path = Path(config.batch_processing.output_folder) / config_filename

try:
    config.save_yaml(config_path)
    print(f"Config saved: {config_path}")
except Exception as e:
    config.save_yaml(config_filename)
    print(f"Config saved: {config_filename}")

## Cleanup

In [ ]:
if 'conn' in locals() and conn is not None:
    conn.close()
    print("OMERO connection closed")

print(f"Total images processed: {len(processed_images) if 'processed_images' in locals() else 0}")
print(f"Table ID: {table_id if 'table_id' in locals() else 'N/A'}")

## Usage Options

```python
# Option 1: Full workflow
table_id, processed_images = pipeline.run_full_workflow()

# Option 2: Split workflow
table_id, images_list = pipeline.create_annotation_table()
table_id, processed_images = pipeline.run_annotation(table_id, images_list)

# Option 3: Resume from existing table
table_id, processed_images = pipeline.run_annotation(existing_table_id)
```